# Modelling Semantic Plausibility with LLAMA 2

---




Note that in order to avoid OutOfMemory error, one might have to restart runtime from section to section.

## 0. Some constants

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Change constants, if needed

In [ ]:
BASE_DIR_PAP = '/content/drive/MyDrive/semantic plausibility/datasets/pap/train-dev-test-split-filtered/binary'
BASE_DIR_PEP = '/content/drive/MyDrive/semantic plausibility/datasets/pep-3k/train-dev-test-split'
CWD = '/content/drive/MyDrive/semantic plausibility/Llama'
RAW = '/content/drive/MyDrive/semantic plausibility/datasets/pap/raw-annotations/dataset.tsv'
CONCRETE = '/content/drive/MyDrive/semantic plausibility/Llama/concrete_13428_2013_403_MOESM1_ESM.xlsx'

In [ ]:
TRAIN_PAP_FN = 'train_pap.csv'
DEV_PAP_FN = 'dev_pap.csv'
TEST_PAP_FN = 'test_pap.csv'
TRAIN_PEP_FN = 'train_pep.csv'
DEV_PEP_FN = 'dev_pep.csv'
TEST_PEP_FN = 'test_pep.csv'

## 1. Experiment 1: Fine-tuning using PAP

## 1.1. Install dependencies

* Skip this if doing inference only

In [ ]:
# 8-bit optimizers and 8-bit inference layers for PyTorch, speed up training and inference
!pip install -q -U bitsandbytes
# access to pretrained models
!pip install -q -U git+https://github.com/huggingface/transformers.git
# parameter-efficient fine-tuning for efficiently adapting large pretrained models to downstream applications
!pip install -q -U git+https://github.com/huggingface/peft.git
# for easy and fast training of transformers models on any distributed setup
!pip install -q -U git+https://github.com/huggingface/accelerate.git
# for easily accessing and sharing datasets
!pip install -q datasets
# For transformer-based reinforcement learning
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 8.7 MB/s eta 0:00:00


Class TL;DR

- `AutoModelForCausalLM`: for causal language modeling tasks, which involve predicting the next token in a sequence.

- `BitsAndBytesConfig`: to configure the quantization settings when loading a model in 8-bit or 4-bit precision12.

- `HfArgumentParser`: for parsing arguments for command-line applications. It is specifically designed to parse dataclasses.

- `TrainingArguments`: to define the training configuration for a model. It includes parameters like learning rate, batch size, number of epochs, etc.

- `AutoTokenizer`: to automatically instantiate a tokenizer from a pre-trained model's name or path.

- `pipeline`: to create a pipeline object for performing a variety of NLP tasks, such as text classification, named entity recognition, and more. It simplifies the process of applying a model to an input.

In [ ]:
# Hugging Face's datasets library for loading and processing datasets
from datasets import load_dataset, Dataset, DatasetDict
# For creating data classes
from dataclasses import dataclass, field
# For type hinting
from typing import Optional
# PyTorch library for tensor computations and deep learning
import torch
from torch.utils.data import DataLoader # to load data examples in batch
# Low-Rank Approximation from PEFT
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
# For creating progress bars
from tqdm import tqdm
# For data manipulation and analysis
import pandas as pd
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline, DataCollatorWithPadding

# from accelerate import Accelerator #for distributed computing
# accelerator = Accelerator()
tqdm.pandas()
from transformers import logging
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)


## 1.2. Global variables

* Skip this part if do inference

#### Load Tokenizer, Model

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf" # non-gated model from HuggingFace hub, not the official gated model from Meta
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

* QLoRA parameters
- `lora_r`: attention dimension (default 64)
- `lora_alpha`: alpha param for LoRA layers (default 16)


In [ ]:
@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default=model_name, metadata={"help": "the model name"})
    dataset_text_field: Optional[str] = field(default="text", metadata={"help": "the text field of the dataset"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    batch_size: Optional[int] = field(default=4, metadata={"help": "the batch size"})
    seq_length: Optional[int] = field(default=512, metadata={"help": "Input sequence length"})
    gradient_accumulation_steps: Optional[int] = field(
        default=2, metadata={"help": "the number of gradient accumulation steps"}
    )
    load_in_8bit: Optional[bool] = field(default=False, metadata={"help": "load the model in 8 bits precision"})
    load_in_4bit: Optional[bool] = field(default=True, metadata={"help": "load the model in 4 bits precision"})
    use_peft: Optional[bool] = field(default=True, metadata={"help": "Whether to use PEFT or not to train adapters"})
    trust_remote_code: Optional[bool] = field(default=True, metadata={"help": "Enable `trust_remote_code`"})
    output_dir: Optional[str] = field(default=f"{CWD}/output", metadata={"help": "the output directory"})
    peft_lora_r: Optional[int] = field(default=64, metadata={"help": "the r parameter of the LoRA adapters"})
    peft_lora_alpha: Optional[int] = field(default=16, metadata={"help": "the alpha parameter of the LoRA adapters"})
    peft_lora_dropout: Optional[float] = field(default=0.1, metadata={"help": "dropout probaility for LoRA adapters"})
    logging_steps: Optional[int] = field(default=10, metadata={"help": "the number of logging steps"})
    use_auth_token: Optional[bool] = field(default=False, metadata={"help": "Use HF auth token to access the model"})
    num_train_epochs: Optional[int] = field(default=2, metadata={"help": "the number of training epochs"})
    max_steps: Optional[int] = field(default=-1, metadata={"help": "the number of training steps"})
    save_steps: Optional[int] = field(
        default=0, metadata={"help": "Number of updates steps before two checkpoint saves"}
    )
    save_total_limit: Optional[int] = field(default=10, metadata={"help": "Limits total number of checkpoints."})
    push_to_hub: Optional[bool] = field(default=False, metadata={"help": "Push the model to HF Hub"})
    hub_model_id: Optional[str] = field(default=None, metadata={"help": "The name of the model on HF Hub"})


script_args = ScriptArguments() # use default configuration from the tutorial

#### Apply quantization when loading pretrained LLAMA model

In [ ]:
if script_args.load_in_8bit and script_args.load_in_4bit:
    raise ValueError("You can't load the model in 8 bits and 4 bits at the same time")
elif script_args.load_in_8bit or script_args.load_in_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=script_args.load_in_8bit, load_in_4bit=script_args.load_in_4bit
    )
    device_map = {"": 0}
    torch_dtype = torch.bfloat16
else:
    device_map = None
    quantization_config = None
    torch_dtype = None

model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=script_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_auth_token=script_args.use_auth_token,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### .0. Generation function for a dataset split

In [ ]:
def remove_answer(example):
  "Define a function to remove the answer of the assistant"
  example['target'] = example['text']
  prompt = example['text'].split('Assistant:')[0].strip() + "\nAssistant: "
  example['text'] = prompt

  return example


In [ ]:
def prepare_split(eval_data_set):
  "take an eval split and transform to data_loader to generate text in parallel"

  eval = eval_data_set.map(remove_answer)
  data_loader = DataLoader(eval, batch_size=16)

  return data_loader

In [ ]:
def generate_explainer(data_loader, max_new_tokens):
  "to generate text in batch"
  model.eval()
  results = []
  for batch in tqdm(data_loader, desc='Generating explanations'):
      # encoding text
      inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
      inputs = inputs.to('cuda')

      # generate explainer
      outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        top_p=0.9,
        do_sample=True,
        )
      # decode the output
      for input_text, target, output in zip(batch['text'], batch['target'], outputs):
        generated_text = tokenizer.decode(output, skip_special_tokens=True)
        results.append({'Input Text': input_text, 'Target': target, 'Generated Text': generated_text})

  return pd.DataFrame(results)

In [ ]:
def generate_and_save_explainer(data_loader, max_new_tokens, output_name):
  split_df = generate_explainer(data_loader, max_new_tokens)
  split_df.to_csv(f'{CWD}/prediction/{output_name}.csv',index=False)

This part is based on this tutorial [link](https://colab.research.google.com/drive/1ggaa2oRFphdBmqIjSEbnb_HGkcIRC2ZB?usp=sharing). The main idea is adapting the dataset to fit the generative model, and post-processing to obtain label after performing text-generation.


### 5.1 Transform the dataset to fit the training format of the generative model

In [ ]:
def prepare_dataset(basedir,question_template):
  train_df = pd.read_csv(f'{basedir}/train.csv')
  dev_df = pd.read_csv(f'{basedir}/dev.csv')
  test_df = pd.read_csv(f'{basedir}/test.csv')
  id_to_label = {1: 'Plausible', 0: 'Implausible'}
  # change text to fit the generative approach
  # Zeroshot instructions
  # ### Human: Categorize the following events as plausibile or implausible. You should only say either 'Plausible' or 'Implausible'.
  # event: broadcast concentrates alignment
  train_df['instruction'] = train_df.apply(lambda row: f"{question_template}\nevent: {row['text']}\nAssistant: {id_to_label[row['label']]}", axis=1)
  dev_df['instruction'] = dev_df.apply(lambda row: f"{question_template}\nevent: {row['text']}\nAssistant: {id_to_label[row['label']]}", axis=1)
  test_df['instruction'] = test_df.apply(lambda row: f"{question_template}\nevent: {row['text']}\nAssistant: {id_to_label[row['label']]}", axis=1)

  ds_train = Dataset.from_dict({"text": train_df["instruction"].tolist(), "label": train_df['label'].tolist()})
  ds_dev = Dataset.from_dict({"text": dev_df["instruction"].tolist(), "label": dev_df['label'].tolist()})
  ds_test = Dataset.from_dict({"text": test_df["instruction"].tolist(), "label": test_df['label'].tolist()})

  instructions_ds_dict = DatasetDict({"train": ds_train, "dev": ds_dev, "test": ds_test})
  return instructions_ds_dict


In [ ]:
# zeroshot prompt
zeroshot_question_template = """
### Human: Categorize the following events as plausibile or implausible. You should only say either 'Plausible' or 'Implausible'.
"""

### 5.2 Finetuning using QLora

* Skip this part if doing evaluation
* Apply LoRA

In [ ]:
def instruction_finetuning(instructions_ds_dict):

  training_args = TrainingArguments(
      output_dir=script_args.output_dir,
      per_device_train_batch_size=script_args.batch_size,
      gradient_accumulation_steps=script_args.gradient_accumulation_steps,
      learning_rate=script_args.learning_rate,
      logging_steps=script_args.logging_steps,
      num_train_epochs=script_args.num_train_epochs,
      max_steps=script_args.max_steps,
      report_to=script_args.log_with,
      save_steps=script_args.save_steps,
      save_total_limit=script_args.save_total_limit,
      push_to_hub=script_args.push_to_hub,
      hub_model_id=script_args.hub_model_id,
  )

  if script_args.use_peft:
      peft_config = LoraConfig(
          r=script_args.peft_lora_r,
          lora_alpha=script_args.peft_lora_alpha,
          bias="none",
          task_type="CAUSAL_LM",
      )
  else:
      peft_config = None

  tokenizer.padding_side = 'right'

  trainer = SFTTrainer(
      model=model,
      args=training_args,
      max_seq_length=script_args.seq_length,
      train_dataset=dataset['train'],
      eval_dataset=dataset['dev'],
      dataset_text_field=script_args.dataset_text_field,
      peft_config=peft_config,
  )
  trainer.train()
  trainer.save_model(training_args.output_dir)

In [ ]:
instructions_ds_dict = prepare_dataset(BASE_DIR_PAP,zeroshot_question_template)
instruction_finetuning(instructions_ds_dict)

* Train (about 30m)

### 5.3 Performance with dev/test

In [ ]:
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
# !pip install trl
# from trl import SFTTrainer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
# Hugging Face's datasets library for loading and processing datasets
from datasets import load_dataset, Dataset, DatasetDict
# For creating data classes
from dataclasses import dataclass, field
# For type hinting
from typing import Optional
# PyTorch library for tensor computations and deep learning
import torch
from torch.utils.data import DataLoader # to load data examples in batch
# Low-Rank Approximation from PEFT
from peft import LoraConfig, PeftModel
# For creating progress bars
from tqdm import tqdm
# For data manipulation and analysis
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
tqdm.pandas()
from transformers import logging
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
instructions_ds_dict = prepare_dataset(BASE_DIR_PAP,zeroshot_question_template)
dev_loader = prepare_split(instructions_ds_dict['dev'])
test_loader = prepare_split(instructions_ds_dict['test'])

Map:   0%|          | 0/173 [00:00<?, ? examples/s]

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf" # non-gated model from HuggingFace, not the official gated model from Meta
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map = {"": 0},
)
new_model = f"{CWD}/output"
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
# should run each cell individually, otherwise OutOfMemoryError
generate_and_save_explainer(dev_loader, 5, "dev_result")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [ ]:
generate_and_save_explainer(test_loader, 5, "test_result")

In [ ]:
import pandas as pd
dev_result = pd.read_csv(f'{CWD}/dev_result.csv')
test_result = pd.read_csv(f'{CWD}/test_result.csv')
dev_result['predict'] = dev_result.apply(lambda row: row["Generated Text"].split("Assistant:")[1].strip(),axis=1)
dev_result['truth'] = dev_result.apply(lambda row: row["Target"].split("Assistant:")[1].strip(),axis=1)
test_result['predict'] = test_result.apply(lambda row: row["Generated Text"].split("Assistant:")[1].strip(),axis=1)
test_result['truth'] = test_result.apply(lambda row: row["Target"].split("Assistant:")[1].strip(),axis=1)

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_curve, auc, accuracy_score
def evaluate_prediction(y_eval, y_pred):
  precision = precision_score(y_eval, y_pred)
  recall = recall_score(y_eval, y_pred)
  accuracy = accuracy_score(y_eval, y_pred)
  print(f'Precision: {precision:.3f} / Recall: {recall:.3f} / Accuracy: {accuracy:.3f}')
  # Compute False Positive Rate, True Positive Rate, and AUC score
  fpr, tpr, thresholds = roc_curve(y_eval, y_pred)
  auc_score = auc(fpr, tpr)
  print(f'AUC: {auc_score:.3f}')

In [ ]:
dev_truth = [1 if "Plausible" == t else 0 for t in dev_result['truth'].tolist()]
dev_pred = [1 if "Plausible" in t else 0 for t in dev_result['predict'].tolist()]

In [ ]:
test_truth = [1 if "Plausible" == t else 0 for t in test_result['truth'].tolist()]
test_pred = [1 if "Plausible" in t else 0 for t in test_result['predict'].tolist()]
evaluate_prediction(test_truth, test_pred)

Precision: 0.662 / Recall: 0.379 / Accuracy: 0.420
AUC: 0.450


In [ ]:
evaluate_prediction(dev_truth, dev_pred)

Precision: 0.689 / Recall: 0.341 / Accuracy: 0.422
AUC: 0.481


## 2. Experiment 2: Finetuning with augmented data

* This part is based on this [tutorial](https://www.datacamp.com/tutorial/fine-tuning-llama-2)

### 2.1 Get the prototypes of each combination - class

In [ ]:
import pandas as pd
import ast

In [ ]:
# use the raw annotation because there are abstract combination info already
raw_df = pd.read_csv(RAW, sep='\t')
# convert the string representation to actual numerical representation
lists = ['rating', 'distribution_multiclass', 'distribution_binary']
raw_df[lists] = raw_df[lists].applymap(lambda x: ast.literal_eval(x.strip()))
# Apply a lambda function to filter based on the condition
filtered_df = raw_df.loc[raw_df['distribution_binary'].apply(lambda x: max(x) > 70)]
# filter out 'unsure' binary datapoints
filtered_df = filtered_df.query("majority_binary != 'unsure'")
# group by abstractness_combinationa and majority_binary. Note that there is only 47 groups but not 2x27=54 groups
grouped_df = filtered_df.groupby(['abstractness_combination','majority_binary'])

In [ ]:
# Define a function to get a random sample from each group
def get_random_datapoint(group):
    return group.sample(1)

# Apply the function to each group
random_datapoints = grouped_df.apply(get_random_datapoint).reset_index(drop=True)

In [ ]:
combi = raw_df['abstractness_combination'].unique() # 27 combi
from itertools import product
# Create a reference DataFrame with all possible combinations
all_combinations = list(product(combi, [0, 1]))  # Assuming 27 types and 2 classes
reference_df = pd.DataFrame(all_combinations, columns=['abstractness_combination', 'majority_binary'])
reference_df

,abstractness_combination,majority_binary
0,a-m-a,0
1,a-m-a,1
2,a-c-m,0
3,a-c-m,1
4,a-c-a,0
5,a-c-a,1
6,a-m-m,0
7,a-m-m,1
8,a-a-a,0
9,a-a-a,1


In [ ]:
# Convert columns in the original DataFrame to int64
random_datapoints['majority_binary'] = random_datapoints['majority_binary'].astype(int)
# Merge the reference DataFrame with original DataFrame
merged_df = pd.merge(reference_df, random_datapoints, how='left', left_on=['abstractness_combination', 'majority_binary'], right_on=['abstractness_combination', 'majority_binary'])

# Identify the missing combinations
missing_combinations = merged_df[merged_df.isnull().any(axis=1)][['abstractness_combination', 'majority_binary']]
# there are 6 combinations that voted as plausible only, but not implausible
print("Missing combinations:")
print(missing_combinations)

Missing combinations:
   abstractness_combination  majority_binary
10                    m-m-a                0
14                    m-m-m                0
18                    c-m-a                0
26                    c-m-c                0
38                    m-c-a                0
42                    m-a-a                0
44                    c-a-c                0


In [ ]:
missing_combi = missing_combinations['abstractness_combination'].tolist()
# Filter rows based on the specified list
filtered_missing_df = raw_df.query("majority_binary != 'unsure'")[raw_df['abstractness_combination'].isin(missing_combi)].loc[raw_df['original_label'].apply(lambda x: x == "implausible")]
# sample 6 combinations originally labelled as `implausible`
additional_grouped_df = filtered_missing_df.groupby(['abstractness_combination'])
# Apply the function to each group
additional_random_datapoints = additional_grouped_df.apply(get_random_datapoint).reset_index(drop=True)

<ipython-input-14-92253044c1cc>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_missing_df = raw_df.query("majority_binary != 'unsure'")[raw_df['abstractness_combination'].isin(missing_combi)].loc[raw_df['original_label'].apply(lambda x: x == "implausible")]


In [ ]:
# combine to the final df (54 combi)
final_df = pd.concat([random_datapoints, additional_random_datapoints], axis = 0)
final_df.to_csv(f'{CWD}/prediction/pap_prototype.csv', index=False)

### 2.2 Generate explainer for prototypes

* Install Dependencies for new session, if needed

* Generate explainer for prototypes

In [ ]:
pap_df = pd.read_csv(f'{CWD}/prediction/pap_prototype.csv')
pap_df['target'] = pap_df.apply(lambda row: f"### Human: Is the event `{row['event']}` plausible?\n### Assistant: {'Plausibile' if row['majority_binary'] == 1 else 'Implausible'} because", axis=1)

In [ ]:
prototype_to_generate = pap_df[['event', 'abstractness_combination','majority_binary','target']]

* Install dependencies, set global variables, generate function

In [ ]:
prompt_template = """
<s>[INST] <<SYS>>
You are careful assistant. Your task is to categorize the following events as plausible or implausible.
You should always start the answer by `Plausible` or `Implausible`. Events could be either asbtract or concrete.
Plausible events could be typical and preferable (e.g. `Kids eat strawberry`),
but a lot plausible events are unlikely, atypical and they should not happen (e.g. `Man eats paintballs).
Implausible events do not make any sense (e.g. `Child eat bridge`).
<</SYS>>
"""

# <s>[INST] <<SYS>>
# {{ system_prompt }}
# <</SYS>>

# {{ user_message }} [/INST]

In [ ]:
prototype_to_generate.head()

,event,abstractness_combination,majority_binary,target
0,lack mitigates disruption,a-a-a,0,### Human: Is the event `lack mitigates disrup...
1,theorist presides association,a-a-a,1,### Human: Is the event `theorist presides ass...
2,conflict entails trousers,a-a-c,0,### Human: Is the event `conflict entails trou...
3,detail invokes letter,a-a-c,1,### Human: Is the event `detail invokes letter...
4,outcome presides part,a-a-m,0,### Human: Is the event `outcome presides part...


In [ ]:
prototype_to_generate['text'] = prototype_to_generate.apply(lambda row: f"{prompt_template}\n{row['target']}", axis=1)
ds_prototype_to_generate = Dataset.from_pandas(prototype_to_generate)
prototype_loader = DataLoader(ds_prototype_to_generate, batch_size=16)
prototype_explainer_df = generate_explainer(prototype_loader, 80)
prototype_explainer_df.to_csv(f"{CWD}/prediction/pap_prototype_explainer.csv",index=False)
# about 4m for about 4 batches

In [ ]:
ds_prototype_to_generate

Dataset({
    features: ['event', 'abstractness_combination', 'majority_binary', 'target', 'text'],
    num_rows: 54
})

We use cleaned explainers (with some formatting and deleting), namely `pap_prototype_explainer_cleaned.xlsx`

### 2.3 Create Plausibility Explainer train-dev dataset

#### 2.3.1. Add `abstractness_combination`

* read a split

In [ ]:
import pandas as pd
train_df = pd.read_csv(f'{BASE_DIR_PAP}/train.csv')
# dev_df = pd.read_csv(f'{BASE_DIR_PAP}/dev.csv')
# test_df = pd.read_csv(f'{BASE_DIR_PAP}/test.csv')

In [ ]:
# read concreteness ratings
conc_df = pd.read_excel(CONCRETE)
# read raw, cleaned annotation
raw_df = pd.read_csv(RAW, sep='\t')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download en_core_web_sm --quiet
# Load the spaCy English language model
import spacy
nlp = spacy.load('en_core_web_sm')

2024-01-14 08:27:24.685951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 08:27:24.686010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 08:27:24.687309: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 08:27:26.040052: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
def assign_abstractness_combination(svo, conc_df):
    """
    Take an event and assign their abstractness combination based on ratings from an external source.
    """
    doc = nlp(svo)
    lemmas = [token.lemma_ for token in doc]
    abstract_score = []

    for l in lemmas:
        conc_value = conc_df.loc[conc_df['Word'] == l, "Conc.M"].values
        if len(conc_value) == 0: # if the word is missing, assuming that it's a proper noun and is highly concrete
            conc_value = [5]
        abstract_score.extend(conc_value)

    abstract_combi =['','','']
    for i, score in enumerate(abstract_score):
        if score <= 2:
            abstract_combi[i] = 'a'
        elif score < 4:
            abstract_combi[i] = 'm'
        else:
            abstract_combi[i] = 'c'

    return abstract_combi

In [ ]:
def add_abstractness_combination(split_df, conc_df, output_name):
  "Take a split dataframe, add abstractness combination and save as a new csv file"
  # merge a split with cleaned raw annotation
  merged_df = split_df.merge(raw_df, how = 'left', left_on='text', right_on='event', suffixes=('_split','_raw'))
  # check if there are datapoints without abstractness combination
  null_abstract_df = merged_df[merged_df['abstractness_combination'].isna()]
  # Apply the function to calculate abstract scores for each row in "text" column
  # This might take a while
  null_abstract_df.loc[:,'abstract_score'] = null_abstract_df['text'].apply(lambda x: assign_abstractness_combination(x, conc_df))
  null_abstract_df['abstractness_combination'] = null_abstract_df['abstract_score'].apply(lambda x: '-'.join(x))
  null_abstract_df.drop(['abstract_score'], axis=1)
  # merge the two dataframes
  dropna_df = merged_df.dropna(subset=['abstractness_combination'])
  final_df = pd.concat([dropna_df, null_abstract_df], axis=0)
  selected_columns = final_df[['text', 'original_label_split', 'label', 'abstractness_combination']]
  selected_columns.to_csv(f'{CWD}/instruction_pap/{output_name}', index=False)

In [ ]:
add_abstractness_combination(train_df, conc_df, 'train_augmented.csv')
# add_abstractness_combination(dev_df, conc_df, 'dev_augmented.csv')
# add_abstractness_combination(test_df, conc_df, 'test_augmented.csv')

<ipython-input-22-400acf169f9e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_abstract_df.loc[:,'abstract_score'] = null_abstract_df['text'].apply(lambda x: assign_abstractness_combination(x, conc_df))
<ipython-input-22-400acf169f9e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_abstract_df['abstractness_combination'] = null_abstract_df['abstract_score'].apply(lambda x: '-'.join(x))


In [ ]:
df = pd.read_csv(f'{CWD}/instruction_pap/train_augmented.csv')
df

,text,original_label_split,label,abstractness_combination
0,group releases album,plausible,1,m-c-c
1,rich unfold interest,implausible,0,m-c-a
2,fruit reduces risk,plausible,1,c-a-a
3,market drives innovation,plausible,1,c-c-a
4,firefighter works schedule,plausible,1,c-c-m
...,...,...,...,...
1381,consensus reads attempt,plausible,1,m-m-m
1382,crash flies characteristic,implausible,1,m-c-m
1383,risk outweighs benefit,plausible,1,a-c-m
1384,aquarium contains chance,implausible,0,c-m-a


#### 2.3.2. Add prompt templates

* group explainers based on abstractness level

We use cleaned explainers (with some formatting and deleting), namely `pap_prototype_explainer_cleaned.xlsx`

In [ ]:
import pandas as pd
prompt_protype = pd.read_excel(f'{CWD}/prediction/pap_prototype_explainer_cleaned.xlsx')
prompt_protype.head()

,event,abstractness_combination,label,generated_explainer
0,lack mitigates disruption,a-a-a,0,Implausible because `lack` is a negative term ...
1,theorist presides association,a-a-a,1,Plausibile because it is a common occurrence f...
2,conflict entails trousers,a-a-c,0,Implausible because `trousers` are not a thing...
3,detail invokes letter,a-a-c,1,Plausibile because the concept of detail and l...
4,outcome presides part,a-a-m,0,Implausible because it is an unlikely and nons...


In [ ]:
def group_prototype(prompt_protype):
  # add Q-A template
  prompt_protype['fewshot_explainer'] = prompt_protype.apply(lambda row: f"### Human: Is the event `{row['event']}` plausible? ### Assistant: {row['generated_explainer']}", axis=1)
  # Group by 'abstractness_combination' and concatenate 'explainer'
  df_grouped = prompt_protype.groupby('abstractness_combination')['fewshot_explainer'].apply(lambda x: '\n\n'.join(x)).reset_index()
  return df_grouped
df_grouped = group_prototype(prompt_protype)
df_grouped.head()

,abstractness_combination,fewshot_explainer
0,a-a-a,### Human: Is the event `lack mitigates disrup...
1,a-a-c,### Human: Is the event `conflict entails trou...
2,a-a-m,### Human: Is the event `outcome presides part...
3,a-c-a,### Human: Is the event `saga injures courtesy...
4,a-c-c,### Human: Is the event `breach accelerates se...


In [ ]:
def add_fewshot_prompt(split_df, df_grouped, output_name):
  # merge the split with grouped prototype explainers
  augmented_split_df = split_df.merge(df_grouped, how='left', left_on=['abstractness_combination'], right_on=['abstractness_combination'], suffixes=('_split','_explainer'))
  # transform event to template
  augmented_split_df.rename(columns={'text':'event'}, inplace=True)
  augmented_split_df.loc[:,'target'] = augmented_split_df.apply(lambda row: f"{'Plausible' if row['label'] == 1 else 'Implausible'}", axis=1)
  augmented_split_df.loc[:,'text'] = augmented_split_df.apply(lambda row: f"{row['fewshot_explainer']}\n\n### Human: Is the event `{row['event']}` plausible? ### Assistant: {'Plausible' if row['label'] == 1 else 'Implausible'} because", axis=1)
  # write to a new csv file
  augmented_split_df[['text','target','event','label']].to_csv(f'{CWD}/instruction_pap/{output_name}',index=False)

In [ ]:
train_df = pd.read_csv(f'{CWD}/instruction_pap/train_augmented.csv')

In [ ]:
add_fewshot_prompt(train_df, df_grouped, 'train_augmented_togenerate.csv')

#### 2.3.3. Generate explainers for PAP train split

In [ ]:
import pandas as pd
train_augmented_togenerate = pd.read_csv(f'{CWD}/instruction_pap/train_augmented_togenerate.csv')
train_augmented_togenerate_sm_df = train_augmented_togenerate[:5] #try with 5 examples


,text,target,event,label
0,### Human: Is the event `wind weaves salmon` p...,Plausible,group releases album,1
1,### Human: Is the event `candidate shows abili...,Implausible,rich unfold interest,0
2,### Human: Is the event `dolphin conducts prof...,Plausible,fruit reduces risk,1
3,### Human: Is the event `video pays homage` pl...,Plausible,market drives innovation,1
4,### Human: Is the event `band ignites photon` ...,Plausible,firefighter works schedule,1


In [ ]:
def generate_explainer_sequentially(text, max_new_tokens):
  "to generate text sequentially"
  model.eval()
  input_ids = tokenizer(text, padding=True, truncation=True, return_tensors="pt").input_ids
  input_ids = input_ids.to('cuda')

  # generate explainer
  outputs = model.generate(
    input_ids,
    max_new_tokens=max_new_tokens,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    top_p=0.9,
    do_sample=True,
    )
  # decode the output
  end_of_input_ids = input_ids.shape[1]
  generated_ids = outputs[0][end_of_input_ids:]
  generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
  result = generated_text
  if '###' in generated_text:
    result = generated_text.split('###')[0]

  return result

In [ ]:
# Apply the function to each row in the 'text' column with a progress bar
tqdm.pandas(desc="Generating explanations")
train_augmented_togenerate_sm_df['explainer'] = train_augmented_togenerate_sm_df['text'].progress_apply(lambda x: generate_explainer_sequentially(x, max_new_tokens=50))


Generating explanations: 100%|██████████| 5/5 [00:31<00:00,  6.22s/it]
<ipython-input-96-d47cc10b0a0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_augmented_togenerate_sm_df['explainer'] = train_augmented_togenerate_sm_df['text'].progress_apply(lambda x: generate_explainer_sequentially(x, max_new_tokens=50))


In [ ]:
train_augmented_togenerate_sm_df # result for the first 5 examples

,text,target,event,label,explainer
0,### Human: Is the event `wind weaves salmon` p...,Plausible,group releases album,1,many groups release albums regularly.\n\n
1,### Human: Is the event `candidate shows abili...,Implausible,rich unfold interest,0,wealthy individuals may have a range of intere...
2,### Human: Is the event `dolphin conducts prof...,Plausible,fruit reduces risk,1,there are several types of fruit that have bee...
3,### Human: Is the event `video pays homage` pl...,Plausible,market drives innovation,1,markets can drive innovation by creating deman...
4,### Human: Is the event `band ignites photon` ...,Plausible,firefighter works schedule,1,it is a typical and common event for a firefig...


In [ ]:
train_augmented_togenerate['explainer'] = train_augmented_togenerate['text'].progress_apply(lambda x: generate_explainer_sequentially(x, max_new_tokens=50))
train_augmented_togenerate.to_csv(f"{CWD}/prediction/train_explainer.csv",index=False)


Generating explanations: 100%|██████████| 1386/1386 [1:51:02<00:00,  4.81s/it]


### 2.4. Finetuning with the Plausibility Explainer dataset

* Only setup constants then skip other parts

#### 2.4.1. Prepare the instruction dataset

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
import pandas as pd
train_explainer_df = pd.read_csv(f"{CWD}/prediction/train_explainer.csv")
train_explainer_df.head()

,text,target,event,label,explainer
0,### Human: Is the event `wind weaves salmon` p...,Plausible,group releases album,1,it is a common and typical event. Many groups ...
1,### Human: Is the event `candidate shows abili...,Implausible,rich unfold interest,0,rich people may have a wide range of interests...
2,### Human: Is the event `dolphin conducts prof...,Plausible,fruit reduces risk,1,"fruit, especially strawberries, are rich in an..."
3,### Human: Is the event `video pays homage` pl...,Plausible,market drives innovation,1,"innovation is often driven by market forces, s..."
4,### Human: Is the event `band ignites photon` ...,Plausible,firefighter works schedule,1,it is a typical and realistic event for a fire...


In [ ]:
train_explainer_df['text'] = train_explainer_df.apply(lambda row: f"### Human: Is the event `{row['event']}` plausible?\n### Assistant: {row['target']} because {row['explainer'].strip()}",axis=1)
dataset = Dataset.from_dict({'text': train_explainer_df['text'].tolist(),'label': train_explainer_df['label'].tolist()})

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1386
})

In [ ]:
dataset[0]['text']

'### Human: Is the event `group releases album` plausible?\n### Assistant: Plausible because it is a common and typical event. Many groups and bands release albums all the time.'

#### 2.4.2 Instruction Finetuning with augmented training data

* Make sure to load Tokenizer and Model from Hugging Face

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model
new_model = f"{CWD}/instruction-tuning/llama-2-7b-chat-pap-explainer"

* create 4-bit quantization with NF4 type configuration using BitsAndBytes.

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

We will now load a model using 4-bit precision with the compute dtype "float16" from Hugging Face for faster training

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Next, we will load the tokenizer from Hugginface and set padding_side to “right” to fix the issue with fp16.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

PEFT parameters
Traditional fine-tuning of pre-trained language models (PLMs) requires updating all of the model's parameters, which is computationally expensive and requires massive amounts of data.

Parameter-Efficient Fine-Tuning (PEFT) works by only updating a small subset of the model's parameters, making it much more efficient. Learn about parameters by reading the PEFT official documentation.

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

Training parameters

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

Model fine-tuning

Supervised fine-tuning (SFT) is a key step in reinforcement learning from human feedback (RLHF). The TRL library from HuggingFace provides an easy-to-use API to create SFT models and train them on your dataset with just a few lines of code. It comes with tools to train language models using reinforcement learning, starting with supervised fine-tuning, then reward modeling, and finally proximal policy optimization (PPO).

We will provide SFT Trainer the model, dataset, Lora configuration, tokenizer, and training parameters.

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1386 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.657700
50,1.061700
75,0.883900
100,0.879700
125,0.859900
150,0.855400
175,0.856200
200,0.858100
225,0.824500
250,0.839100


('/content/drive/MyDrive/semantic plausibility/PAP/instruction-tuning/llama-2-7b-chat-pap-explainer/tokenizer_config.json',
 '/content/drive/MyDrive/semantic plausibility/PAP/instruction-tuning/llama-2-7b-chat-pap-explainer/special_tokens_map.json',
 '/content/drive/MyDrive/semantic plausibility/PAP/instruction-tuning/llama-2-7b-chat-pap-explainer/tokenizer.json')

Evaluation

In [ ]:
dev_df = pd.read_csv(f"{BASE_DIR_PAP}/dev.csv")
test_df = pd.read_csv(f"{BASE_DIR_PAP}/test.csv")
dev_df['prompt'] = dev_df.apply(lambda row: f"### Human: Is the event `{row['text']}` plausible?\n### Assistant: ",axis=1)
test_df['prompt'] = test_df.apply(lambda row: f"### Human: Is the event `{row['text']}` plausible?\n### Assistant: ",axis=1)

In [ ]:
dev_df

,text,original_label,label,prompt
0,press shakes rent,implausible,0,### Human: Is the event `press shakes rent` pl...
1,pair pronounces validation,implausible,1,### Human: Is the event `pair pronounces valid...
2,caper extracts finger,implausible,0,### Human: Is the event `caper extracts finger...
3,motorway forbids distribution,implausible,1,### Human: Is the event `motorway forbids dist...
4,amendment establishes wall,plausible,1,### Human: Is the event `amendment establishes...
...,...,...,...,...
168,exit publicizes war,implausible,1,### Human: Is the event `exit publicizes war` ...
169,moon severs debut,implausible,0,### Human: Is the event `moon severs debut` pl...
170,municipality decorates street,plausible,1,### Human: Is the event `municipality decorate...
171,regiment contributes personnel,plausible,1,### Human: Is the event `regiment contributes ...


In [ ]:
def generate_explainer_sequentially(text, max_new_tokens):
  "to generate text sequentially"
  model.eval()
  input_ids = tokenizer(text, padding=True, truncation=True, return_tensors="pt").input_ids
  input_ids = input_ids.to('cuda')

  # generate explainer
  outputs = model.generate(
    input_ids,
    max_new_tokens=max_new_tokens,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    top_p=0.9,
    do_sample=True,
    )
  # decode the output
  end_of_input_ids = input_ids.shape[1]
  generated_ids = outputs[0][end_of_input_ids:]
  generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
  result = generated_text
  if '###' in generated_text:
    result = generated_text.split('###')[0]

  return result

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="Generating explanations")
dev_df['explainer'] = dev_df['prompt'].progress_apply(lambda x: generate_explainer_sequentially(x, max_new_tokens=50))
dev_df.to_csv(f"{CWD}/prediction/dev_explainer.csv",index=False)

Generating explanations:   0%|          | 0/173 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Generating explanations: 100%|██████████| 173/173 [1:34:00<00:00, 32.60s/it]


In [ ]:
test_df['explainer'] = test_df['prompt'].progress_apply(lambda x: generate_explainer_sequentially(x, max_new_tokens=50))
test_df.to_csv(f"{CWD}/prediction/test_explainer.csv",index=False)

Generating explanations: 100%|██████████| 174/174 [1:34:23<00:00, 32.55s/it]


In [ ]:
dev_result = pd.read_csv(f'{CWD}/prediction/dev_explainer.csv')
test_result = pd.read_csv(f'{CWD}/prediction/test_explainer.csv')

In [ ]:
dev_result['predict'] = dev_result.apply(lambda row: row["explainer"].split()[0].strip(),axis=1)
test_result['predict'] = test_result.apply(lambda row: row["explainer"].split()[0].strip(),axis=1)

In [ ]:
dev_result

,text,original_label,label,prompt,explainer,predict
0,press shakes rent,implausible,0,### Human: Is the event `press shakes rent` pl...,Implausible because rent is a financial term a...,Implausible
1,pair pronounces validation,implausible,1,### Human: Is the event `pair pronounces valid...,Implausible because a pair of shoes or an inan...,Implausible
2,caper extracts finger,implausible,0,### Human: Is the event `caper extracts finger...,Implausible because it is not possible to extr...,Implausible
3,motorway forbids distribution,implausible,1,### Human: Is the event `motorway forbids dist...,Implausible because motorways are not capable ...,Implausible
4,amendment establishes wall,plausible,1,### Human: Is the event `amendment establishes...,Implausible because amending a law or regulati...,Implausible
...,...,...,...,...,...,...
168,exit publicizes war,implausible,1,### Human: Is the event `exit publicizes war` ...,Implausible because war is a global event that...,Implausible
169,moon severs debut,implausible,0,### Human: Is the event `moon severs debut` pl...,Implausible because the moon is a celestial bo...,Implausible
170,municipality decorates street,plausible,1,### Human: Is the event `municipality decorate...,Plausible because it is a common practice for ...,Plausible
171,regiment contributes personnel,plausible,1,### Human: Is the event `regiment contributes ...,Implausible because a regiment is a military u...,Implausible


In [ ]:
dev_truth = dev_result['label'].tolist()
dev_pred = [1 if "Plausible" in t else 0 for t in dev_result['predict'].tolist()]
evaluate_prediction(dev_truth, dev_pred)

Precision: 0.747 / Recall: 0.553 / Accuracy: 0.549
AUC: 0.546


In [ ]:
test_truth = test_result['label'].tolist()
test_pred = [1 if "Plausible" in t else 0 for t in test_result['predict'].tolist()]
evaluate_prediction(test_truth, test_pred)

Precision: 0.764 / Recall: 0.548 / Accuracy: 0.557
AUC: 0.564


#### 2.4.3 Inference with PEP

In [ ]:
!pip install -q accelerate peft bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import pandas as pd
from tqdm import tqdm

In [ ]:
def generate_explainer_sequentially(text, max_new_tokens):
  "to generate text sequentially"
  model.eval()
  input_ids = tokenizer(text, padding=True, truncation=True, return_tensors="pt").input_ids
  input_ids = input_ids.to('cuda')

  # generate explainer
  outputs = model.generate(
    input_ids,
    max_new_tokens=max_new_tokens,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    top_p=0.9,
    do_sample=True,
    )
  # decode the output
  end_of_input_ids = input_ids.shape[1]
  generated_ids = outputs[0][end_of_input_ids:]
  generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
  result = generated_text
  if '###' in generated_text:
    result = generated_text.split('###')[0]

  return result

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf" # non-gated model from HuggingFace, not the official gated model from Meta
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# compute_dtype = getattr(torch, "float16")

# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=False,
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=quant_config,
#     device_map={"": 0}
# )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map = {"": 0},
)
new_model = f"{CWD}/instruction-tuning/llama-2-7b-chat-pap-explainer"
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
inference_text = [
    "scientists make art",
    "dog eats aeroplane",
    "man creates inspiration",
    "illusion boosts performance"
]

In [ ]:
ps = [f'### Human: Is the event `{t}` plausible? \n ### Assistant: ' for t in inference_text]

In [ ]:
for p in ps:
  print(p, "\n", generate_explainer_sequentially (p, 50))

event ### Human: Is the event `scientists make art` plausible? 
 ### Assistant:  
 Implausible because scientists are not typically known for creating art, but rather for conducting scientific research and experiments. While some scientists may have artistic hobbies or interests, it is unlikely that they would be creating art as their primary
event ### Human: Is the event `dog eats aeroplane` plausible? 
 ### Assistant:  
 Implausible because dogs cannot eat aeroplanes. Aeroplanes are man-made objects and dogs do not have the ability to consume them. It is also not a common occurrence for dogs to eat objects that are not meant to be
event ### Human: Is the event `man creates inspiration` plausible? 
 ### Assistant:  
 Plausible because it is a common occurrence for people to create something that inspires others. This could be a work of art, a piece of music, a literary work, or even an invention. The event is likely to happen in the
event ### Human: Is the event `illusion boosts perfo

In [ ]:
pep_test = pd.read_csv(f"{BASE_DIR_PEP}/test.csv")
pep_test.head()

,label,text
0,1,worm enter cave
1,1,elephant toss cat
2,1,beak tap purse
3,1,wolf push cup
4,0,pen etch oil


In [ ]:
pep_test["prompt"] = pep_test.apply(lambda row: f"### Human: Is the event `{row['text']}` plausible? \nAssistant:", axis=1)

In [ ]:
# Apply the function to each row in the 'text' column with a progress bar
tqdm.pandas(desc="Generating explanations")
pep_test['explainer'] = pep_test['prompt'].progress_apply(lambda x: generate_explainer_sequentially(x, max_new_tokens=50))

Generating explanations: 100%|██████████| 307/307 [16:37<00:00,  3.25s/it]


In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_curve, auc, accuracy_score
def evaluate_prediction(y_eval, y_pred):
  precision = precision_score(y_eval, y_pred)
  recall = recall_score(y_eval, y_pred)
  accuracy = accuracy_score(y_eval, y_pred)
  print(f'Precision: {precision:.3f} / Recall: {recall:.3f} / Accuracy: {accuracy:.3f}')
  # Compute False Positive Rate, True Positive Rate, and AUC score
  fpr, tpr, thresholds = roc_curve(y_eval, y_pred)
  auc_score = auc(fpr, tpr)
  print(f'AUC: {auc_score:.3f}')

In [ ]:
pep_test['predict'] = pep_test.apply(lambda row: row["explainer"].split()[0].strip(),axis=1)

In [ ]:
test_pred = [1 if "Plausible" in t else 0 for t in pep_test['predict'].tolist()]
evaluate_prediction(pep_test['label'].tolist(),test_pred)

Precision: 0.500 / Recall: 0.346 / Accuracy: 0.502
AUC: 0.501
